In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# @IMPORT-MERGE
import os
os.environ["MPLCONFIGDIR"] = "/tmp"
import numpy as np
import pandas as pd
import itertools
import cv2
import random
from IPython.display import HTML, display
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter, df_to_radmat
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs, circle_locs, sigproc_v2_im
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.utils import data
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.utils import data
from plaster.tools.calibration.calibration import Calibration
from plaster.tools.ipynb_helpers.displays import hd, movie
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/abbe8_2t")
run = job.runs[0]
n_channels = run.sigproc_v2.n_channels
n_cycles = run.sigproc_v2.n_cycles

# Calibration

In [ ]:
# Show Calibration information
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    psfs = run.sigproc_v2.params.calibration.psfs(ch_i=ch_i)

    with z(_noaxes=True):
        display("Below:")
        with z(_cols=6):
            n_zs = psfs.shape[0]
            for z_i in range(0, n_zs // 2):
                plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)

        display("Most in Focus:")
        z_i = n_zs // 2
        plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z, _noaxes=True, _notools=True)

        display("Above:")
        with z(_cols=6):
            for z_i in range(n_zs // 2 + 1, n_zs):
                plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)
                
    # Show the Calibration illumination balance
    illum = np.array(run.sigproc_v2.params.calibration[f"regional_illumination_balance.instrument_channel[{ch_i}]"])
    z.im(1.0 / illum, f_title="Illumination map")

# Quality

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    qdf = run.ims_import.qualities()
    quality = qdf[qdf.channel_i == ch_i].sort_values(["quality"])
    z.hist(quality.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution channel {ch_i}")

    row_iz = utils.ispace(0, len(qdf), 3)

    # COMBINE all images for common percentile calculations
    ims = np.concatenate([
        run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i].flatten()
        for row in qdf.iloc[row_iz].itertuples()
    ])
    bot, top = np.percentile(ims, (50, 99.99))

    # SHOW example of worst, median, and best all using the same cspan
    hd("h3", f"Examples of frames by quality")
    with z(_cols=3, _cspan=(bot, top)):
        names = ("worst", "median", "best")
        for name, row in zip(names, qdf.iloc[row_iz].itertuples()):
            z.im(run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i], f_title=f"Channel: {ch_i} {name}")            

# Alignment

In [ ]:
field_df = run.sigproc_v2.fields().copy()
field_df["alignment"] = np.sqrt(field_df.aln_x**2 + field_df.aln_y**2)
alignment = field_df.groupby("field_i").alignment.max().values
z.cols(alignment, f_x_axis_label="field_i", f_y_axis_label="n_pixels", f_title="Max. alignment dist.")
good_field_iz = np.argwhere(alignment < 50)

# Filters

In [ ]:
df = run.sigproc_v2.fields__n_peaks__peaks__radmat()
filt_df = df_filter(df, fields=good_field_iz)

# SNR

In [ ]:
with z(_cols=4):
    snr = df_to_radmat(filt_df, radmat_field="snr")
    top = np.percentile(snr, 97)
    
    for ch_i in range(run.ims_import.n_channels):
        z.hist(
            snr[:, ch_i, :], _bins=(0, top, 200),
            f_y_axis_label="count", f_x_axis_label="SNR",
            f_title=f"SNR distribution ch_i={ch_i}"
        )

In [ ]:
a = filt_df.groupby("field_i").mean()[["snr"]]
b = pd.DataFrame(dict(field_i=np.arange(225))).set_index("field_i")
z.cols(a.join(b, how="right").values.flatten(), _size_x=800, f_title="Mean SNR by field")

# Unfiltered signal by field

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Unfiltered Channel {ch_i}")

    sig = df_to_radmat(df, channel_i=ch_i)
    
    _df = df.groupby(["field_i", "cycle_i"]).signal.mean().reset_index()
    sig_by_field_cycle = (
        pd.pivot_table(
            _df, values="signal", index="field_i", columns=["cycle_i"]
        )
        .reset_index()
        .rename_axis(None, axis=1)
        .drop(columns="field_i")
    ).values

    top = np.percentile(sig, 99)

    with z(_cols=3):
        z.hist(
            sig, _bins=(0, top, 200),
            f_y_axis_label="count", f_x_axis_label="Signal",
            f_title=f"Signal distribution (all fields)"
        )
        z.im_clus(
            sig_by_field_cycle,
            f_y_axis_label="field_i", f_x_axis_label="cycle_i",
            f_title=f"Mean sig. by field and cycle",
            _cspan=(0, top),
        )

        z.im_clus(sig, _cspan=(0, top), f_title=f"radmat sample")

# Unfiltered signal by cycle

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    sig = run.sigproc_v2.sig()[:, ch_i, :]
    center = np.median(sig[:, 0])
    n_cycles = run.sigproc_v2.n_cycles
    max_x = np.percentile(sig, 99)
    bins = np.linspace(-1000, max_x, 200)
    _hist, _ = np.histogram(sig[:, 0], bins=bins)
    max_y = np.max(_hist)
    with z(_cols=5, _size=180, _noaxes=True, _bins=bins, _range=(0, max_x, 0, max_y*1.2)):
        for cy_i in range(n_cycles):
            _sig = sig[:, cy_i].flatten()
            with z(_merge=True, f_title=f"cy_i={cy_i}"):
                z.hist(_sig)
                z.line(x=[center, center], y=[0, max_y], color="red")
                z.line(x=[0, 0], y=[0, max_y], color="black")


# Image Visualization

In [ ]:
# Pick a peak_filter set with df_filter
#filt_df = df_filter(df, fields=good_field_iz, dark=1500, on_through_cy_i=10, off_at_cy_i=59)
filt_df = df_filter(df, fields=good_field_iz, min_aspect_ratio=1.2, dark=1500, on_through_cy_i=2)
filt_df

In [ ]:
def sigproc_v2_movie_on_peaks(df, channel_i=0, field_i=None):
    """
    If field_i is None then it is enforced that all peaks in the df must come form one field
    """
    if field_i is not None:
        df = df[df.field_i == field_i]
        
    assert df.field_i.nunique() == 1
    field_i = df.field_i[0]
    
    ims = run.sigproc_v2.aln_ims[field_i, channel_i, :]

    overlay = np.zeros((ims.shape[-2:]), dtype=np.uint8)
    locs = df[["aln_y", "aln_x"]].drop_duplicates().values
    overlay = 255 * circle_locs(overlay, locs, fill_mode="one", inner_radius=4, outer_radius=5).astype(np.uint8)

    movie(ims, overlay, _cper=(50, 99.9), _duration=1, _labels=[
        f"aligned & balanced fl_i:{field_i} ch_i:{channel_i} cy_i: {cy_i}"
        for cy_i in range(ims.shape[0])
    ])


sigproc_v2_movie_on_peaks(filt_df, field_i=0)


In [ ]:
# View a field, channel, cycle with roll-over properties for debugging

# Change this line to select a different (field, channel, cycle) to view:
field_channel_cycle = (0, 0, 0)

mea = run.ims_import.dim
sigproc_v2_im(run, *field_channel_cycle, _size=max(800,mea), _range=(0, mea, 0, mea))

# TODO MAKE A FILT_DF VERSION OF THIS

# Wizards

In [ ]:
plots.wizard_scat_df(run, channel_i=0)

In [ ]:
plots.wizard_xy_df(run, channel_i=0)

In [ ]:
plots.wizard_raw_images(run, show_circles=False, peak_i_square=True, square_radius=7)